In [ ]:
import nltk
import numpy as np
import random
import string # to process standard python strings
import os, json
import pandas as pd

In [ ]:
# Get question and nested answer 
data = []
with open('data/climatology-1.json') as json_file:
    data = json.load(json_file)
count = 1

def traverseAnswerObjects(answersObjects):
    print("answer: " + answersObjects["text"])
    print("\n")
    if(len(answersObjects["answers"])):
        for answerObject in answersObjects["answers"]:
            traverseAnswerObjects(answerObject)

for questionObject in data:
    print("Question: "+ questionObject["question"])
    print("\n")
    traverseAnswerObjects(questionObject)

    count += 1
    if(count == 2):
        break

In [ ]:
# Get question and first answer
data = []
with open('data/climatology-1.json') as json_file:
    data = json.load(json_file)
columns = ['question', 'answer']
questionAndAnswerDf = pd.DataFrame(columns=columns)
for index, questionObject in enumerate(data):
    questionAndAnswerDf.loc[index] = [questionObject["question"], questionObject["text"]]

In [ ]:
questionAndAnswerDf.head()